# 文本分块

文本分块的核心原理是将加载后的长篇文档，切分成更小、更易于处理的单元。

这些被切分出的文本块，是后续向量检索和模型处理的基本单位。

## 重要性

将长文本分解为适当大小的片段，主要基于两大考量：

- 模型的上下文限制

- 检索生成的性能需求。

### 满足模型上下文控制

将文本分块的首要原因，是为了适应RAG系统中两个核心组件的硬性限制：

- 嵌入模型：嵌入模型负责将文本块转换为向量。这类模型有严格的输入长度上限。任何超出此限制的文本块在输入时都会被截断，导致信息丢失，生成的向量也无法完整代表原文的语义。因此，文本块的大小必须小于等于嵌入模型的上下文窗口。

- 大语言模型：负责根据检索到的上下文生成答案。LLM同样有上下文窗口限制，检索到的所有文本块，连同用户问题和提示词，都必须能被放入这个窗口中。


### 为何“块”不是越大越好

#### 嵌入过程中的信息损失

多数嵌入模型都是基于 Transformer 架构，其工作流程大致如下：

1. 分词 (Tokenization): 将输入的文本块分解成一个个 token。

2. 向量化 (Vectorization): Transformer 为每个 token 生成一个高维向量表示。

3. 池化 (Pooling): 通过某种方法将所有 token 的向量压缩成一个**单一的向量**，这个向量代表了整个文本块的语义。

压缩过程中，信息损失是不可避免的。

文本块越长，包含的语义点越多，这个单一向量所承载的信息就越稀释，导致其表示变得笼统，关键细节被模糊化，从而降低了检索的精度。

#### 生成过程的“大海捞针” (Lost in the Middle)

当LLM处理非常长的、充满大量信息的上下文时，它倾向于更好地记住开头和结尾的信息，而忽略中间部分的内容。

如果提供给LLM的上下文块又大又杂，充满了与问题无关的噪音，模型就很难从中提取出最关键的信息来形成答案，从而导致回答质量下降或产生幻觉。

#### 主题稀释导致检索失败

一个好的文本块应该聚焦于一个明确、单一的主题。如果一个块包含太多不相关的主题，它的语义就会被稀释，导致在检索时无法被精确匹配。

## 基础分块策略

LangChain 提供了丰富且易于使用的文本分割器（Text Splitters），下面将介绍几种最核心的策略。

### 固定大小分块

CharacterTextSplitter 将文本按段落进行分割，分割后的段落依次合并。该方法会监控累积长度，当超过 chunk_size 时形成新块，并通过重叠机制（chunk_overlap）保持上下文连续性。

但实际实现上并非严格的固定大小分块。

- 优先保持段落完整性：只有当添加新段落会导致总长度超过 chunk_size 时，才会结束当前块。

- 处理超长段落：如果单个段落超过 chunk_size，系统会发出警告但仍将其作为完整块保留。

因此，CharacterTextSplitter 更准确地应该称为"段落感知的自适应分块"，块大小会根据段落边界动态调整。

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/蜂医.txt")
docs = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=200,    # 每个块的目标大小为100个字符
    chunk_overlap=10   # 每个块之间重叠10个字符，以缓解语义割裂
)

chunks = text_splitter.split_documents(docs)

print(f"文本被切分为 {len(chunks)} 个块。\n")
print("--- 前5个块内容示例 ---")
for i, chunk in enumerate(chunks[:5]):
    print("=" * 60)
    # chunk 是一个 Document 对象，需要访问它的 .page_content 属性来获取文本
    print(f'块 {i+1} (长度: {len(chunk.page_content)}): "{chunk.page_content}"')


Created a chunk of size 201, which is longer than the specified 200
Created a chunk of size 277, which is longer than the specified 200
Created a chunk of size 296, which is longer than the specified 200


文本被切分为 14 个块。

--- 前5个块内容示例 ---
块 1 (长度: 72): "# 蜂医

游戏《三角洲行动》中的支援型干员

蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。"
块 2 (长度: 201): "蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果。 \[1-2]****"
块 3 (长度: 189): "* 中文名

  罗伊•斯米

* 外文名

  Roy smee \[2]**

* 别    名

  罗伊、蜂医

* 性    别

  男

- 登场作品

  [三角洲行动](/item/%E4%B8%89%E8%A7%92%E6%B4%B2%E8%A1%8C%E5%8A%A8/63251542?fromModule=lemma_inlink)

- 生    日"
块 4 (长度: 133): "- 生    日

  2008年2月23日 \[3]**

- 身    高

  176 cm \[3]**

- 体    重

  75 kg \[3]**

## 目录

1. 1[角色设定](#1)
2. 2[角色定位](#2)
3. 3[技能](#3)"
块 5 (长度: 189): "1) ▪[战术装备 - 激素枪](#3-1)
2) ▪[战术道具 - 烟幕无人机](#3-2)
3) ▪[战术道具 - 蜂巢科技烟雾弹](#3-3)

1. ▪[干员特长 - 高效救援](#3-4)

## 角色设定

三角洲行动：医疗角色蜂医！让你不再为血量担忧

蜂医是游戏中的一名战地医生，拥有丰富的参军履历。

蜂医在干员档案中标明他有一个妻子和女儿。

## 角色定位"


###  递归字符分块

RecursiveCharacterTextSplitter 通过分隔符层级递归处理，相对与固定大小分块，改善了超长文本的处理效果。

分隔符：["\n\n", "\n", " ", ""]

分块器首先尝试使用最高优先级的分隔符（如段落标记）来切分文本。如果切分后的块仍然过大，会继续对这个大块应用下一优先级分隔符（如句号），如此循环往复，直到块满足大小限制。

算法流程：

1. 寻找有效分隔符: 从分隔符列表中从前到后遍历，找到第一个在当前文本中存在的分隔符。如果都不存在，使用最后一个分隔符（通常是空字符串 ""）。

2. 使用选定的分隔符切分文本，然后遍历所有片段

    - 如果片段不超过块大小: 暂存到 _good_splits 中，准备合并

    - 如果片段超过块大小：

        - 首先，将暂存的合格片段通过 _merge_splits 合并成块。

        - 然后，检查是否还有剩余分隔符：
  
          - 有剩余分隔符: 递归调用 _split_text 继续分割。

          - 无剩余分隔符: 直接保留为超长块。

固定大小分块遇到超长段落时只能发出警告并保留，递归分块会继续使用更细粒度的分隔符（句子→单词→字符）直到满足大小要求。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/蜂医.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "。", "，", " ", ""],  # 分隔符优先级
    chunk_size=200,
    chunk_overlap=10,
)

chunks = text_splitter.split_documents(docs)


此外， RecursiveCharacterTextSplitter 能够针对特定的编程语言（如Python, Java等）使用预设的、更符合代码结构的分隔符。

它们通常包含语言的顶级语法结构（如类、函数定义）和次级结构（如控制流语句），以实现更符合代码逻辑的分割。

### 语义分块

在语义主题发生显著变化的地方进行切分，使每个分块都具有高度的内部语义一致性。

实现原理：

1. 句子分割: 使用标准的句子分割规则（例如，基于句号、问号、感叹号）将输入文本拆分成一个句子列表。

2. 上下文感知嵌入: 对于列表中的每一个句子，将其与前后 n 个句子组合起来，然后对这个临时的、更长的组合文本进行嵌入。每个句子最终得到的嵌入向量就融入了其上下文的语义。

3. 计算语义距离：计算每对相邻句子的嵌入向量之间的余弦距离。这个距离值量化了两个句子之间的语义差异——距离越大，表示语义关联越弱，跳跃越明显。

4. 识别断点：分析所有计算出的距离值，并根据一个统计方法来确定一个动态阈值。所有距离大于此阈值的点，都被识别为语义上的“断点”。

5. 合并成块：根据识别出的所有断点位置，将原始的句子序列进行切分，并将每个切分后的部分内的所有句子合并起来，形成一个最终的、语义连贯的文本块。

### 基于文档结构的分块

对于具有明确结构标记的文档格式（如Markdown、HTML、LaTex），可以利用其内部的结构标记（如标题、段落、列表等）来进行分块。